In [3]:
print("ok")

ok


In [4]:
%pwd

'/Users/chaitanyagarg/Desktop/VS Code /Medical-Chatbot-/research'

In [5]:
import os 
os.chdir("../")

In [6]:
%pwd

'/Users/chaitanyagarg/Desktop/VS Code /Medical-Chatbot-'

In [7]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

/Users/chaitanyagarg/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/chaitanyagarg/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# Extra Data from PDF File 
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)
    documents=loader.load()

    return documents

In [9]:
extracted_data = load_pdf_file(data='Data/')

In [10]:
# extracted_data

In [11]:
#Split the Data into Text Chunks

def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks 

In [12]:
text_chunks=text_split(extracted_data)
print("Legnth of Text Chunks :- ",len(text_chunks))

Legnth of Text Chunks :-  5699


In [13]:
text_chunks

[Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-28T15:38:25-05:00', 'moddate': '2005-05-04T13:53:15-06:00', 'author': 'Clifford', 'enhanced': 'By PDF Enhancer 2.5/Win', 'spdf': '1096', 'source': 'Data/Book.pdf', 'total_pages': 599, 'page': 0, 'page_label': '1'}, page_content='The GALE\nENCYCLOPEDIA of\nAlTERNATIVE\nMEDICINE\nSECOND EDITION\nGEAMv2htp/tp  10/11/04  10:45 AM  Page 1'),
 Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-28T15:38:25-05:00', 'moddate': '2005-05-04T13:53:15-06:00', 'author': 'Clifford', 'enhanced': 'By PDF Enhancer 2.5/Win', 'spdf': '1096', 'source': 'Data/Book.pdf', 'total_pages': 599, 'page': 1, 'page_label': '2'}, page_content='The GALE\nENCYCLOPEDIA of\nAlternative\nMEDICINE\nSECOND EDITION\nJACQUELINE L. LONGE, PROJECT EDITOR\nVOLUME\nD-K\n2\nGEAMv2htp/tp  10/11/04  10:45 AM  Page 3'),
 Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)',

In [14]:
from langchain_community.embeddings import HuggingFaceEmbeddings

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )
    return embeddings


In [15]:
embeddings = download_hugging_face_embeddings()

/var/folders/4t/xdcnp0wd31d_4444pnrlq4dr0000gn/T/ipykernel_20318/3457686428.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [16]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [17]:
# query_result

In [18]:
from dotenv import load_dotenv
load_dotenv()

True

In [19]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [20]:
from pinecone import Pinecone, ServerlessSpec
import os

pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

index_name = "medicalbot"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
    print("Index created!")
else:
    print("Index already exists")


Index already exists


In [21]:
import os 
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [22]:
# Embed each chunk and upsert the embeddings into your Pinecone index.

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    embedding=embeddings,
    index_name=index_name
)

In [23]:
# Load Existing index

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embedding into Pinecone index 
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [24]:
docsearch

In [25]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [26]:
retrieved_docs = retriever.invoke("What is Acne?")

In [27]:
retrieved_docs

[Document(id='d0342462-9a4a-45ab-983b-84a8d19b0d6a', metadata={'author': 'Clifford', 'creationdate': '2004-12-28T15:38:25-05:00', 'creator': 'PyPDF', 'enhanced': 'By PDF Enhancer 2.5/Win', 'moddate': '2005-05-04T13:53:15-06:00', 'page': 4.0, 'page_label': '5', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data/Book.pdf', 'spdf': '1096', 'total_pages': 599.0}, page_content='A\nAbscess\nAcidophilus\nAcne\nAconite\nAcupressure\nAcupuncture\nAdemetionine\nAdie’s pupil\nAfrican pygeum\nAgastache\nAging\nAIDS\nAlcoholism\nAlexander technique\nAlfalfa\nAlisma\nAllergies\nAllium cepa\nAloe\nAlpha-hydroxy\nAlzheimer’s disease\nAmino acids\nAndrographis\nAndrostenedione\nAnemarrhena\nAnemia\nAngelica root\nAngina\nAnise\nAnkylosing spondylitis\nAnorexia nervosa\nAnthroposophical medicine\nAntioxidants\nAnxiety\nBates method\nBayberry\nBedsores\nBedwetting\nBee pollen\nBehavioral optometry\nBehavioral therapy'),
 Document(id='7bedd1c7-2952-44fb-9749-2e95aa4b3fa3', metadata={'author': 'Cliffo

In [28]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [29]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "use the following pieces of retrieved context to answer"
    "the question. If you don't know the answer, say that you "
    "don't know. use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [30]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain= create_retrieval_chain(retriever, question_answer_chain)

In [31]:
response = rag_chain.invoke({"input": "What is Acne?"})
print(response["answer"])

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}